# Код для получения информации об административных округах расположения станций

In [ ]:
%pip install beautifulsoup4 requests

In [1]:
import requests
import bs4

In [3]:
main_page_html = requests.get("https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/")
main_page_bs = bs4.BeautifulSoup(main_page_html.text, "html.parser")

In [14]:
districts_list = main_page_bs.find("ul", {"class": "dropdown-menu"})
districts = []
district_links = {}
for ch_cont in districts_list.children:
    if isinstance(ch_cont, bs4.NavigableString):
        continue
    ch = ch_cont.findChild("a")
    if len(ch.text) == 0:
        continue
    districts.append(ch.text)
    onclick = ch.get("onclick")
    link = onclick.split("=")[1].split("'")[1]
    district_links[ch.text] = link
districts
district_links

{'Восточный': 'https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/vostochnyj.html',
 'Западный': 'https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/zapadnyj.html',
 'Зеленоградский': 'https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/zelenogradskij.html',
 'Новомосковский': 'https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/novomoskovskij.html',
 'Северный': 'https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/severnyj.html',
 'Северо-Восточный': 'https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/severo-vostochnyj.html',
 'Северо-Западный': 'https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/severo-zapadnyj.html',
 'Центральный': 'https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/centralnyj.html',
 'Юго-Восточный': 'https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/jugo-vostochnyj.html',
 'Юго-Западный': 'https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/jugo-zapadnyj.html',
 'Южный': 'https://metro.mwmoskva.ru/stancii-spisok/po-okrugam/juzhnyj.html'}

In [20]:
station_districts = {}
for district_name in districts:
    district_link = district_links[district_name]
    district_html = requests.get(district_link)
    district_bs = bs4.BeautifulSoup(district_html.text, "html.parser")
    district_stations = set()
    for station_cont in district_bs.find_all("div", {"class": "list-name"}):
        district_stations.add(station_cont.text.strip())
    for station in district_stations:
        station_districts[station] = district_name
station_districts

{'Выхино': 'Юго-Восточный',
 'Соколиная Гора': 'Восточный',
 'Партизанская': 'Восточный',
 'Сокольники': 'Восточный',
 'Шоссе Энтузиастов': 'Восточный',
 'Перово': 'Восточный',
 'Улица Дмитриевского': 'Восточный',
 'Преображенская площадь': 'Восточный',
 'Андроновка': 'Юго-Восточный',
 'Авиамоторная': 'Юго-Восточный',
 'Первомайская': 'Восточный',
 'Бульвар Рокоссовского': 'Восточный',
 'Новокосино': 'Восточный',
 'Щелковская': 'Восточный',
 'Локомотив': 'Восточный',
 'Измайлово': 'Восточный',
 'Черкизовская': 'Восточный',
 'Измайловская': 'Восточный',
 'Кусково': 'Восточный',
 'Новогиреево': 'Восточный',
 'Семеновская': 'Восточный',
 'Лухмановская': 'Восточный',
 'Белокаменная': 'Восточный',
 'Электрозаводская': 'Центральный',
 'Очаково': 'Западный',
 'Проспект Вернадского': 'Западный',
 'Юго-Западная': 'Западный',
 'Пионерская': 'Западный',
 'Давыдково': 'Западный',
 'Тропарёво': 'Западный',
 'Озёрная': 'Западный',
 'Багратионовская': 'Западный',
 'Аэропорт Внуково': 'Западный',
 'Ку

In [24]:
import pandas as pd

df = pd.DataFrame({"Name": station_districts.keys(), "District": station_districts.values()})
df

,Name,District
0,Выхино,Юго-Восточный
1,Соколиная Гора,Восточный
2,Партизанская,Восточный
3,Сокольники,Восточный
4,Шоссе Энтузиастов,Восточный
...,...,...
297,Шаболовская,Южный
298,Ленинский проспект,Южный
299,Коломенская,Южный
300,Шипиловская,Южный


In [ ]:
df.to_csv("station_distr.csv", index=False)